In [1]:
import pandas as pd
import numpy as np
import vaex

In [2]:
traf = vaex.open('../../TRAFICO/traf_hist_unir.hdf5')

traf.info()


In [3]:
#read acci_filtered_vaex

acci_filtered_vaex = vaex.open('../../acci_filtered_vaex.hdf5')

acci_filtered_vaex.info()

column,type,unit,description,expression
fecha,str,,,
station,str,,,
tmed,str,,,
prec,str,,,
velmedia,str,,,
hora,str,,,
distrito,str,,,
localizacion,str,,,
coordenada_x_utm,float64,,,
coordenada_y_utm,float64,,,


In [4]:
acci_fil = acci_filtered_vaex[['idelem', 'lesividad']]

In [5]:
#from vaex to pandas

acci_fil = acci_fil.to_pandas_df()

In [6]:
#drop duplicates by idelem

acci_fil = acci_fil.drop_duplicates(subset='idelem', keep='first')

In [7]:
#acci_fil to vaex

acci_fil_vaex = vaex.from_pandas(acci_fil)

In [8]:
#left join acci_fil with traf

traf_acci = traf.join(acci_fil_vaex, on='idelem', how='left')

In [9]:
traf_acci.info()

In [10]:
#drop missing in lesividad column

traf_acci2 = traf_acci.dropmissing(column_names=['lesividad'])

In [11]:
#drop lesividad column

traf_acci2 = traf_acci2.drop(['lesividad'])

In [12]:
#read acci_filtered_vaex

acci_filtered_vaex = vaex.open('../../acci_filtered_vaex.hdf5')

In [13]:
# to pandas

acci = acci_filtered_vaex.to_pandas_df()

In [14]:
acci['accidente']  = 1

In [15]:
#rename idelem to id

acci.rename(columns={'idelem': 'id'}, inplace=True)

In [16]:
acci.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112833 entries, 0 to 112832
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   fecha             112833 non-null  object 
 1   station           112833 non-null  object 
 2   tmed              109458 non-null  object 
 3   prec              112082 non-null  object 
 4   velmedia          82934 non-null   object 
 5   hora              112833 non-null  object 
 6   distrito          112833 non-null  object 
 7   localizacion      112833 non-null  object 
 8   coordenada_x_utm  112833 non-null  float64
 9   coordenada_y_utm  112833 non-null  float64
 10  lesividad         112833 non-null  object 
 11  meteo             112833 non-null  object 
 12  latitud           112833 non-null  float64
 13  longitud          112833 non-null  float64
 14  id                112833 non-null  object 
 15  accidente         112833 non-null  int64  
dtypes: float64(4), int64

In [17]:
#filter accis by id equal to diff

acci = acci[['fecha', 'hora', 'id', 'accidente']]

#add 0 to the left of hora if length is 4

for i in range(len(acci)):
    if len(acci['hora'][i]) == 4:
        acci['hora'][i] = '0' + acci['hora'][i]

acci

C:\Users\calba\AppData\Local\Temp\ipykernel_8552\1723448084.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acci['hora'][i] = '0' + acci['hora'][i]


,fecha,hora,id,accidente
0,2013-01-01,19:00,6404,1
1,2013-01-01,00:00,4608,1
2,2013-01-01,00:00,4579,1
3,2013-01-01,13:00,4516,1
4,2013-01-01,17:00,5632,1
...,...,...,...,...
112828,2022-12-31,05:00,10554,1
112829,2022-12-31,06:00,6947,1
112830,2022-12-31,06:00,4006,1
112831,2022-12-31,06:30,10785,1


In [18]:
#acci to vaex

acci_vaex = vaex.from_pandas(acci)

acci_vaex['unir'] = acci_vaex['fecha'] + ' ' + acci_vaex['hora'] + ':00' + ' ' + acci_vaex['id']
acci_vaex = acci_vaex.drop(['fecha','hora','id'])

#group by unir and sum

acci_vaex = acci_vaex.groupby('unir', agg={'accidente':'sum'})

acci_vaex

#,unir,accidente
0,2014-11-11 14:30:00 3463,1
1,2014-11-12 16:30:00 10635,1
2,2014-11-12 12:00:00 6019,1
3,2014-11-13 20:00:00 3931,1
4,2014-11-14 13:30:00 4219,1
...,...,...
"109,476",2021-12-23 17:30:00 6747,1
"109,477",2021-12-23 22:30:00 5827,1
"109,478",2021-12-24 10:00:00 3695,1
"109,479",2021-12-25 10:00:00 4080,1


In [19]:
acci_vaex.accidente.value_counts()

1    106146
2      3325
4         7
3         3
dtype: int64

In [20]:
#join traf_acci2 with acci_vaex

traf_acci3 = traf_acci2.join(acci_vaex, on='unir', how='left')

In [21]:
traf_acci3

#,idelem,fecha_full,intensidad,ocupacion,carga,prec,velmedia,unir,accidente
0,3564,2013-01-20 20:30:00,8.0,0.0,1.0,"0,7","3,3",2013-01-20 20:30:00 3564,--
1,3882,2013-01-20 20:30:00,1164.0,4.0,22.0,"0,7","3,3",2013-01-20 20:30:00 3882,--
2,3883,2013-01-20 20:30:00,1143.0,4.0,22.0,"0,7","3,3",2013-01-20 20:30:00 3883,--
3,3884,2013-01-20 20:30:00,958.0,3.0,18.0,"0,7","3,3",2013-01-20 20:30:00 3884,--
4,3885,2013-01-20 20:30:00,1028.0,4.0,20.0,"0,7","3,3",2013-01-20 20:30:00 3885,--
...,...,...,...,...,...,...,...,...,...
"526,371,788",11057,2022-12-31 21:00:00,227.0,1.0,5.0,"0,1","0,3",2022-12-31 21:00:00 11057,--
"526,371,789",11057,2022-12-31 21:30:00,188.0,1.0,5.0,"0,1","0,3",2022-12-31 21:30:00 11057,--
"526,371,790",11057,2022-12-31 22:00:00,96.0,0.0,2.0,"0,1","0,3",2022-12-31 22:00:00 11057,--
"526,371,791",11057,2022-12-31 23:00:00,53.0,0.0,0.0,"0,1","0,3",2022-12-31 23:00:00 11057,--


In [22]:
traf_acci3.accidente.value_counts()

missing    526289366
1              79912
2               2506
4                  6
3                  3
dtype: int64

In [23]:
traf_acci3 = traf_acci3.drop(['unir'])

In [24]:
traf_acci3.info()

In [28]:
traf_acci4 = traf_acci3.drop(['velmedia'])

In [29]:
traf_acci4

#,idelem,fecha_full,intensidad,ocupacion,carga,prec,accidente
0,3564,2013-01-20 20:30:00,8.0,0.0,1.0,"0,7",--
1,3882,2013-01-20 20:30:00,1164.0,4.0,22.0,"0,7",--
2,3883,2013-01-20 20:30:00,1143.0,4.0,22.0,"0,7",--
3,3884,2013-01-20 20:30:00,958.0,3.0,18.0,"0,7",--
4,3885,2013-01-20 20:30:00,1028.0,4.0,20.0,"0,7",--
...,...,...,...,...,...,...,...
"526,371,788",11057,2022-12-31 21:00:00,227.0,1.0,5.0,"0,1",--
"526,371,789",11057,2022-12-31 21:30:00,188.0,1.0,5.0,"0,1",--
"526,371,790",11057,2022-12-31 22:00:00,96.0,0.0,2.0,"0,1",--
"526,371,791",11057,2022-12-31 23:00:00,53.0,0.0,0.0,"0,1",--


In [25]:
#fill missing values in accidente column with 0

#traf_acci3['accidente'] = traf_acci3['accidente'].fillmissing(0)


In [30]:
#split the df in 2

traf_acci4_1 = traf_acci4[:300000000]

traf_acci4_2 = traf_acci4[300000000:]



In [33]:
traf_acci4_1

In [32]:
#save both dfs

#traf_acci4_1.export_hdf5('../../traf_acci4_1.hdf5', progress=True)

export(hdf5) [----------------------------------------]  0.00% estimated time: unknown                 

KeyboardInterrupt: 

In [ ]:
traf_acci4_2.export_hdf5('../../traf_acci4_2.hdf5')

In [27]:
#traf.accidente.value_counts()